In [1]:
%pip install yfinance numpy scipy
import yfinance as yf
import numpy as np
from scipy.stats import norm
from datetime import datetime


Note: you may need to restart the kernel to use updated packages.


In [2]:
# Black-Scholes Greeks
def calculate_greeks(S, K, T, r, sigma, option_type):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if option_type.lower() == 'call':
        delta = norm.cdf(d1)
        theta = (-S * norm.pdf(d1) * sigma / (2 * np.sqrt(T))) - r * K * np.exp(-r * T) * norm.cdf(d2)
        rho = K * T * np.exp(-r * T) * norm.cdf(d2)
    elif option_type.lower() == 'put':
        delta = norm.cdf(d1) - 1
        theta = (-S * norm.pdf(d1) * sigma / (2 * np.sqrt(T))) + r * K * np.exp(-r * T) * norm.cdf(-d2)
        rho = -K * T * np.exp(-r * T) * norm.cdf(-d2)
    else:
        raise ValueError("Option type must be 'call' or 'put'")

    gamma = norm.pdf(d1) / (S * sigma * np.sqrt(T))
    vega = S * norm.pdf(d1) * np.sqrt(T)

    return {
        'Delta': delta,
        'Gamma': gamma,
        'Theta': theta,
        'Vega': vega / 100,   # scaled for 1% change
        'Rho': rho / 100      # scaled for 1% change
    }

# Get user input
ticker = input("Enter a stock ticker: ").upper()
option_type = input("Option type (call or put): ").lower()

stock = yf.Ticker(ticker)
spot_price = stock.history(period="1d")['Close'].iloc[-1]

# Show available expiration dates
options = stock.options
print(f"Available expiration dates: {options}")
exp_date = input("Pick expiration date from above: ")

# Choose a strike
option_chain = stock.option_chain(exp_date)
chain = option_chain.calls if option_type == 'call' else option_chain.puts
print(chain[['strike', 'lastPrice', 'impliedVolatility']].head())

strike = float(input("Pick a strike from the list above: "))
option_row = chain[chain['strike'] == strike].iloc[0]

# Inputs for BSM
K = strike
S = spot_price
T = (datetime.strptime(exp_date, "%Y-%m-%d") - datetime.today()).days / 365
r = 0.05  # risk-free rate (can be dynamic)
sigma = option_row['impliedVolatility']

# Calculate Greeks
greeks = calculate_greeks(S, K, T, r, sigma, option_type)
print(f"\nGreeks for {ticker} {option_type.upper()} option (Strike: {K}, Exp: {exp_date})")
for greek, value in greeks.items():
    print(f"{greek}: {value:.4f}")


Enter a stock ticker:  SPY
Option type (call or put):  Call


Available expiration dates: ('2025-07-08', '2025-07-09', '2025-07-10', '2025-07-11', '2025-07-14', '2025-07-15', '2025-07-16', '2025-07-17', '2025-07-18', '2025-07-25', '2025-07-31', '2025-08-01', '2025-08-08', '2025-08-15', '2025-08-22', '2025-08-29', '2025-09-19', '2025-09-30', '2025-10-17', '2025-10-31', '2025-11-21', '2025-11-28', '2025-12-19', '2025-12-31', '2026-01-16', '2026-03-20', '2026-03-31', '2026-06-18', '2026-06-30', '2026-09-18', '2026-12-18', '2027-01-15', '2027-12-17')


Pick expiration date from above:  2025-10-31


   strike  lastPrice  impliedVolatility
0   370.0     242.92           0.577763
1   380.0     221.95           0.558049
2   385.0     221.03           0.549992
3   390.0     213.65           0.540837
4   400.0     203.80           0.522405


Pick a strike from the list above:  390



Greeks for SPY CALL option (Strike: 390.0, Exp: 2025-10-31)
Delta: 0.9593
Gamma: 0.0005
Theta: -44.0148
Vega: 0.3032
Rho: 1.1095
